In [5]:
import numpy as np

class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        self.x = x
        self.y = y
        
        return x * y
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        
        return dx, dy # 순서 기억에 유의하자.
    
apple = 100
appleNum = 2
tax = 1.1

mul_apple = MulLayer()
mul_apple_tax = MulLayer()

apple_price = mul_apple.forward(apple, appleNum)
total_price = mul_apple_tax.forward(apple_price, tax)

print(total_price)

dout = 1
dapple, dtax = mul_apple_tax.backward(dout)
dapple, dnum = mul_apple.backward(dout)

print(dapple, dnum)
    
class AddLayer:
    def __init__(self):
        pass
    
    def forward(self, x, y):
        return x+y
    def backward(self, dout):
        return dout, dout
    
class Relu():
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x<=0)
        out = x.copy()
        out[self.mask] = 0

        return out

    def backward(self, dout):
        dx = dout.copy()
        dx[self.mask] = 0

        return dx

class Sigmoid():
    def __init__(self):
        self.out = None
        
    def forward(self, x):
        EMIN = -np.log(np.finfo(type(0.1)).max)
        xSafe = np.array(np.maximum(x, EMIN))
        self.out = (1.0 / (1+np.exp(-xSafe))) # overflow나지 않도록 수정함
        
        return self.out
        
    def backward(self, dout):
        dx = dout * self.out * (1 - self.out)
        return dx
    
class Affine:
    def __init__(self, W, b):
        self.W, self.b = W, b
        self.x, self.dW, self.db = None, None, None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis = 0)
        
        return dx
    
class SoftmaxWLoss:
    def __init__(self, W):
        self.loss = None  
        self.y, self.t = None, None
    
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        
        return self.loss
    
    def backward(self, dout = 1):
        # 배치 사용한 경우 배치크기 만큼 나누어주자.
        batch_size = self.t.shape[0] 
        dx = (self.y - self.t) / batch_size
        
        return dx
    
class momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.momentum = momentum
        self.v = None
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for key, val in params.items():
                self.v[key] = np.zeros_like(val)
                
            for key in params.keys():
                self.v[key] = self.momentum * self.v[key] - lr * grads[key]
                params[key] += self.v[key]
                


220.00000000000003
2 100
